In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e26/sample_submission.csv
/kaggle/input/playground-series-s3e26/train.csv
/kaggle/input/playground-series-s3e26/test.csv


## Load in datasets

In [2]:
train=pd.read_csv('/kaggle/input/playground-series-s3e26/train.csv')
test=pd.read_csv('/kaggle/input/playground-series-s3e26/test.csv')

## Initial EDA

Initial shape of test and training data provided as well as columns to work with.

In [3]:
print(train.shape)
train.columns

(7905, 20)


Index(['id', 'N_Days', 'Drug', 'Age', 'Sex', 'Ascites', 'Hepatomegaly',
       'Spiders', 'Edema', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper',
       'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin',
       'Stage', 'Status'],
      dtype='object')

In [4]:
train.describe(include='all')

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
count,7905.000000,7905.000000,7905,7905.000000,7905,7905,7905,7905,7905,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905
unique,NaN,NaN,2,NaN,2,2,2,2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
top,NaN,NaN,Placebo,NaN,F,N,Y,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C
freq,NaN,NaN,4010,NaN,7336,7525,4042,5966,7161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4965
mean,3952.000000,2030.173308,NaN,18373.146490,NaN,NaN,NaN,NaN,NaN,2.594485,350.561923,3.548323,83.902846,1816.745250,114.604602,115.340164,265.228969,10.629462,3.032511,NaN
std,2282.121272,1094.233744,NaN,3679.958739,NaN,NaN,NaN,NaN,NaN,3.812960,195.379344,0.346171,75.899266,1903.750657,48.790945,52.530402,87.465579,0.781735,0.866511,NaN
min,0.000000,41.000000,NaN,9598.000000,NaN,NaN,NaN,NaN,NaN,0.300000,120.000000,1.960000,4.000000,289.000000,26.350000,33.000000,62.000000,9.000000,1.000000,NaN
25%,1976.000000,1230.000000,NaN,15574.000000,NaN,NaN,NaN,NaN,NaN,0.700000,248.000000,3.350000,39.000000,834.000000,75.950000,84.000000,211.000000,10.000000,2.000000,NaN
50%,3952.000000,1831.000000,NaN,18713.000000,NaN,NaN,NaN,NaN,NaN,1.100000,298.000000,3.580000,63.000000,1181.000000,108.500000,104.000000,265.000000,10.600000,3.000000,NaN
75%,5928.000000,2689.000000,NaN,20684.000000,NaN,NaN,NaN,NaN,NaN,3.000000,390.000000,3.770000,102.000000,1857.000000,137.950000,139.000000,316.000000,11.000000,4.000000,NaN


In [5]:
train.isna().sum()

id               0
N_Days           0
Drug             0
Age              0
Sex              0
Ascites          0
Hepatomegaly     0
Spiders          0
Edema            0
Bilirubin        0
Cholesterol      0
Albumin          0
Copper           0
Alk_Phos         0
SGOT             0
Tryglicerides    0
Platelets        0
Prothrombin      0
Stage            0
Status           0
dtype: int64

Identifying categories: Sex, Age (bins?), Stage, Status (target), Drug, Ascites, Hepatomegaly,Spiders and Edema. 

There are multiple things to consider here - catagorical variables and some catagorical variables that are ordinal. 

In [6]:
x=train.drop(axis='columns',labels='Status').dtypes
x.value_counts()
y=x[x.values == "object"]
y.index

Index(['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema'], dtype='object')

In [7]:
for column in y.index.values:
    with open('/kaggle/working/datalog.log', 'w') as logfile:
        logfile.write(f'Train:{train[column].value_counts()/train[column].count()}')
        logfile.write(f'Test:{test[column].value_counts()/test[column].count()}')